In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install shapely
!pip install rasterio
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 31.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached rasterio-1.3.6-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (20.1 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 1.6 MB/s eta 0:00:00
  Using cached cligj-0.7.2-py3-none-any.whl (7.1 kB)
  Using cached click_plugins-1.1.1-py2.py3-none-any.whl (7.5 kB)
  Using cached snuggs-1.4.7-py3-none-any.whl (5.4 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 8.8 MB/s eta 0:00:00
  Using cached affine-2.4.0-py3-none-any.whl (15 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 10.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-whe

In [ ]:
import os
import shapely
from shapely.geometry.multipolygon import MultiPolygon
from shapely import wkt
from shapely import Polygon
import numpy as np
import pandas as pd
import rasterio as rio
import geopandas

In [ ]:
%cd ./gdrive/MyDrive/Projet\ -\ IFT6759/ift6759_trees/idtrees_competition_evaluation

# !git clone https://github.com/NIST-NEON-DSE/idtrees_competition_evaluation.git

/content/gdrive/.shortcut-targets-by-id/184ezBWFOS9bf045eM8oKF-ABBTVfvo38/ift6759_trees/idtrees_competition_evaluation


# Conda/Requirements Installation 
__RUN ALL UNTIL HERE THEN RERUN EVERYTHING BUT THIS__

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:22
🔁 Restarting kernel...


In [ ]:
!conda env create -f environment.yaml
!conda activate idtrees


EnvironmentFileNotFound: '/content/environment.yaml' file not found


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.




In [ ]:
!which conda
# devrait afficher : /usr/local/bin/conda

/usr/local/bin/conda


# Evaluation


## File Paths

In [ ]:
# CLEM YOLOV7
model_name='yolov7'
run_name = 'ensemble03'

raw_exp_label_path = f'../{model_name}/runs/detect/{run_name}/labels/'

In [ ]:
%ls ../yolov7/runs/detect/augment/labels

In [ ]:
# Anni FasterCNN
yolo=False
model_name = 'fastercnn'
run_name = 'RGB_baseline_thres0.5'

raw_exp_label_path = f'../fastercnn-pytorch-training-pipeline/outputs/inference/{run_name}'

In [ ]:
%ls ../fastercnn-pytorch-training-pipeline/outputs/inference/RGB_baseline_thres0.5

MLBS_20.jpg  MLBS_25.txt  MLBS_39.out  MLBS_9.jpg   OSBS_26.txt  OSBS_35.out
MLBS_20.out  MLBS_34.jpg  MLBS_39.txt  MLBS_9.out   OSBS_32.jpg  OSBS_35.txt
MLBS_20.txt  MLBS_34.out  MLBS_42.jpg  MLBS_9.txt   OSBS_32.out
MLBS_25.jpg  MLBS_34.txt  MLBS_42.out  OSBS_26.jpg  OSBS_32.txt
MLBS_25.out  MLBS_39.jpg  MLBS_42.txt  OSBS_26.out  OSBS_35.jpg


In [ ]:
# # Xavier Yolov5
model_name = 'yolov5'
run_name = 'exp27'

raw_exp_label_path = f'../{model_name}/runs/detect/{run_name}/labels/'

In [ ]:
chm_path = '../data/raw/RemoteSensing/CHM'
rgb_path = '../data/raw/RemoteSensing/RGB'
test_im_path = '../processed_data/test/images'
out_dir = f'./runs/clean_labels/{model_name}/'
test_rgb = f'../processed_data/test/images'
raw_true_label_path = f'../processed_data/test/labels'

## Convert Labels

In [ ]:
def yolo_format_to_multipolygon(raw_labels:str, bounds, model_name):
  assert model_name in ['fastercnn', 'yolov5', 'yolov7']
  
  list_boxes = raw_labels.split('\n')
  border_list = []
  poly_list = []

  true_wid = bounds[2] - bounds[0]
  true_hei = bounds[3] - bounds[1]
  
  for l in list_boxes:
    elts = l.split(' ') 
    if len(elts) ==5 or len(elts)==6:  
    # FasterCNN: x_min, y_min, x_max, y_max
    # YOLOV7: Class, x_center, y_center, width, and heigh
      coords = [float(coord) for coord in elts[1:]]
      coords[0] = coords[0] * true_wid + bounds[0]
      coords[2] = coords[2] * true_wid
      coords[1] = coords[1] * true_hei + bounds[1]
      coords[3] = coords[3] * true_hei
      x_l = coords[0] - coords[2]/2
      x_r = coords[0] + coords[2]/2
      y_t = coords[1] - coords[3]/2
      y_b = coords[1] + coords[3]/2

    elif len(elts) == 4:
      coords = [float(coord) for coord in elts]
      x_l = coords[0] * 10 + bounds[0]
      x_r = coords[2]*10 + bounds[0]
      y_t = coords[1]*10 + bounds[1]
      y_b = coords[3]*10 + bounds[1]
    else:
      continue

      
    border_list += [(x_l, y_t), (x_r, y_t), (x_l, y_b), (x_r, y_b), (x_l, y_t)]

    polygon = Polygon([(x_l, y_t), (x_r, y_t), (x_l, y_b), (x_r, y_b), (x_l, y_t)])
    multi = MultiPolygon([polygon])
    
    poly_list.append(multi)
  return poly_list

In [ ]:
for rgb_fn in os.listdir(test_rgb):
  im_n = rgb_fn.split('.')[0] 
  label_fn = im_n + '.txt'

  if not(label_fn in os.listdir(raw_exp_label_path)):
    print(f'{label_fn} not in labels')
    pd.DataFrame(columns=['WKT', 'id']).to_csv(os.path.join(out_dir,'submission',im_n+'_ground.csv'))
    pd.DataFrame(columns=['WKT', 'id']).to_csv(os.path.join(out_dir,'submission',im_n+'_submission.csv'))
    continue

  print(f'opening {label_fn}')
  ras_path = os.path.join(rgb_path,im_n + '.tif')
  raster = rio.open(ras_path)

  with open(os.path.join(raw_exp_label_path, label_fn)) as f:
    raw_exp_labels = f.read()
    submission_list = yolo_format_to_multipolygon(raw_exp_labels, raster.bounds, model_name)


  with open(os.path.join(raw_true_label_path, label_fn)) as f:
    true_labels = f.read()
    true_list = yolo_format_to_multipolygon(true_labels, raster.bounds, model_name)

  print('submission label', len(submission_list))
  print('true label', len(true_list))

  ids = list(range(0, len(submission_list)+len(true_list)))
  submission_arr = pd.DataFrame(shapely.to_wkt(submission_list), columns=['WKT'])
  submission_arr['WKT'] = submission_arr['WKT'].apply(wkt.loads)
  # gdf = gpd.GeoDataFrame(df, crs='epsg:4326')
  submission_arr['id'] = submission_arr.index
  submission_arr.to_csv(os.path.join(out_dir+'submission',im_n+'_submission.csv'))

  true_labels_arr = pd.DataFrame(shapely.to_wkt(true_list), columns=['WKT'])
  true_labels_arr['id'] = true_labels_arr.index
  true_labels_arr.to_csv(os.path.join(out_dir+'submission',im_n+'_ground.csv'))

opening MLBS_9.txt
3772
304
submission label 47
true label 12
opening MLBS_42.txt
3517
386
submission label 44
true label 15
opening MLBS_39.txt
3577
404
submission label 45
true label 16
opening MLBS_20.txt
3191
329
submission label 40
true label 13
opening OSBS_32.txt
3190
310
submission label 40
true label 12
opening OSBS_26.txt
4227
371
submission label 53
true label 15
opening OSBS_35.txt
3861
90
submission label 48
true label 4
opening MLBS_25.txt
3126
602
submission label 39
true label 23
opening MLBS_34.txt
3365
618
submission label 42
true label 24


## Run Evaluation

In [ ]:
from parameters import *
import evaluation
from importlib import reload
import geopandas

reload(evaluation)
clean = f'{out_dir}'
args = evaluation_parameters(['--datadir',clean,'--task','task1','--outputdir',f'./runs/{model_name}/', '--outputfn',f'{run_name}'])
evaluation.run_segmentation_evaluation(args)

./runs/clean_labels/yolov7/submission/MLBS_42_submission.csv

name MLBS_42.tif
Number of detections:  44


Processing each detection for plot MLBS_42.tif: 100%|##########| 44/44 [00:02<00:00, 21.40it/s]

./runs/clean_labels/yolov7/submission/OSBS_32_submission.csv

name OSBS_32.tif
Number of detections:  40


Processing each detection for plot OSBS_32.tif:  95%|#########5| 38/40 [00:01<00:00, 27.68it/s]

./runs/clean_labels/yolov7/submission/MLBS_34_submission.csv

name MLBS_34.tif
Number of detections:  42


...

Processing each detection for plot MLBS_39.tif: 100%|##########| 45/45 [00:01<00:00, 29.57it/s]

./runs/clean_labels/yolov7/submission/OSBS_35_submission.csv

name OSBS_35.tif
Number of detections:  48


Processing each detection for plot OSBS_35.tif:  94%|#########3| 45/48 [00:01<00:00, 34.38it/s]

./runs/clean_labels/yolov7/submission/MLBS_20_submission.csv

name MLBS_20.tif
Number of detections:  40


(array([0.01822076, 0.6983297 , 0.99307289, 0.09588331, 0.47950162,
        0.41353838, 0.689436  , 0.07703727, 0.84138505, 0.81134223,
        0.19971537, 0.81846856, 0.97257886, 0.01335881, 0.32344476,
        0.38456188, 0.89814062, 0.719477  , 0.47385695, 0.77411199,
        0.76781972, 0.68002889, 0.17786364, 0.57018709, 0.5415767 ,
        0.70847578, 0.28710291, 0.29446643, 0.5514226 , 0.26981758,
        0.11479633, 0.53271661, 0.91265852, 0.99227121, 0.81024207,
        0.91931973, 0.34753149, 0.91554848, 0.        , 0.53659064,
        0.        , 0.81853097, 0.11541878, 0.29203918, 0.66046818,
        0.3402972 , 0.68869347, 0.08012682, 0.07644015, 0.71446457,
        0.1975014 , 0.31104618, 0.5363514 , 0.16144827, 0.72127283,
        0.31870387, 0.96177106, 0.20196573, 0.3883446 , 0.14261888,
        0.18406923, 0.79959408, 0.21689713, 0.18448618, 0.31295663,
        0.24672959, 0.10955717, 0.0340576 , 0.09135135, 0.72300706,
        0.0729264 , 0.68280048, 0.42133901, 0.70

# Mean calculations

In [ ]:
metrics = pd.read_csv(f'./runs/{model_name}/{run_name}.csv')

In [ ]:
metrics.mean()

Unnamed: 0    66.500000
itc_id         8.022388
rand_index     0.452231
IoU            0.326165
dtype: float64